In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from datetime import datetime
import time
import os
import sys
from multiprocessing import Process, Event, shared_memory
from multiprocessing import synchronize, Manager
from multiprocessing.managers import DictProxy
from typing import Optional
# import threading

In [ ]:
import cantools
from pprint import pprint
import can
# from can import Message
from cantools.database import Message
from can.interface import Bus  # virtual interface for testing
# from can.interfaces.udp_multicast.bus import GeneralPurposeUdpMulticastBus as GPUDPMCBus
# from can.interfaces.udp_multicast import UdpMulticastBus

In [ ]:
db = cantools.database.load_file('../res/motohawk_new.dbc')
db.messages
example_message = db.get_message_by_name('ExampleMessage')
example_message.signals
# pprint(example_message.__dict__)
example_message.frame_id
example_message.signal_groups
example_message.signal_tree

[message('ExampleMessage', 0x1f0, False, 8, {None: 'Example message used as template in MotoHawk models.'}),
 message('NewMessage', 0x254, False, 8, {None: 'self made message'})]

[signal('Enable', 7, 1, 'big_endian', False, None, 1, 0, None, None, '-', False, None, {0: 'Disabled', 1: 'Enabled'}, None, None),
 signal('AverageRadius', 6, 6, 'big_endian', False, None, 0.1, 0, 0, 5, 'm', False, None, None, None, None),
 signal('Temperature', 0, 12, 'big_endian', True, None, 0.01, 250, 229.52, 270.47, 'degK', False, None, None, None, None)]

496

[]

['Enable', 'AverageRadius', 'Temperature']

In [ ]:
!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo modprobe vcan

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



In [ ]:
!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link add dev vcan0 type vcan
# sshpass -v -p asdf sudo ip link add dev vcan0 type vcan

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.


SSHPASS: read: 

SSHPASS: read: RTNETLINK answers: File exists



In [ ]:
!ip link show vcan0

23: vcan0: <NOARP,UP,LOWER_UP> mtu 72 qdisc noqueue state UNKNOWN mode DEFAULT group default qlen 1000
    link/can 


In [ ]:
# !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set vcan0 type vcan bitrate 500000  # vcan Does not SUPPORT set bitrate on command line!
# !sshpass -p asdf sudo ip link add dev vcan0 type vcan

In [ ]:
# !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set up vcan0
!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set vcan0 up type vcan
# !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set vcan0 up type vcan  bitrate 500000  # vcan Does not SUPPORT set bitrate on command line!
# !sshpass -v -p asdf sudo ip link set up vcan0

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.


SSHPASS: read: 



In [ ]:
bus = Bus(bustype='socketcan', channel='vcan0', bitrate=250000) # interface='virtual'
# bus = GPUDPMCBus(group=UdpMulticastBus.DEFAULT_GROUP_IPv6, port=43113, hop_limit=1)
# bus = UdpMulticastBus(group=UdpMulticastBus.DEFAULT_GROUP_IPv6)
data = example_message.encode({'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1})
message_to_send = can.Message(arbitration_id=example_message.frame_id, data=data, is_extended_id=False) 	
message_to_send
bus
bus.fileno()
# bus.protocol
message_to_send.data
message_to_send.arbitration_id


can.Message(timestamp=0.0, arbitration_id=0x1f0, is_extended_id=False, dlc=8, data=[0xc0, 0x1, 0x40, 0x0, 0x0, 0x0, 0x0, 0x0])

<can.interfaces.socketcan.socketcan.SocketcanBus>

76

bytearray(b'\xc0\x01@\x00\x00\x00\x00\x00')

496

In [ ]:
manager = Manager()
message_proxy = manager.dict()

In [ ]:
# def receive_message(message_proxy: DictProxy, bus_channel:str='vcan0', bitrate:int=25000):
def receive_message(message_proxy: DictProxy, bus:Bus):
	print('entering receive subprocess')
	# bus = Bus(bustype='socketcan', channel=bus_channel, bitrate=bitrate)
	# os.setsid()
	print(f'bus: {bus.fileno()} {bus}')
	msg:Message = bus.recv()
	print(f'message received: {msg}')
	message_proxy['timestamp'] = msg.timestamp
	message_proxy['arbitration_id'] = msg.arbitration_id
	message_proxy['data']=msg.data

In [ ]:
# def send_message(message: can.Message, done: synchronize.Event, bus_channel:str='vcan0', bitrate:int=25000, timeout: Optional[float]=None)->None:
def send_message(message: can.Message, done: synchronize.Event, bus:Bus, timeout: Optional[float]=None)->None:
	print("entering send subprocess")
	# os.setsid()  # create new process group, become session leader, otherwise a process in a same session could not receive signal from the main process
	os.setpgrp()
	print((os.getpid(), os.getsid(os.getpid())))
	# bus = Bus(bustype='socketcan', channel=bus_channel, bitrate=bitrate)
	print(f'bus: {bus.fileno()}, {bus}')
	bus.send(message)
	print('message sent and waiting')
	start = time.time_ns()
	done.wait()
	elapsed = (time.time_ns()-start)//1e6
	print(f'elapsed time: {elapsed:.3f}ms')

In [ ]:

os.getcwd()
bus.fileno()
bus
os.getpid(),os.getsid(os.getpid())

'/home/n/devel/anting_remote/eos/tspace/candycan/nbs'

76

<can.interfaces.socketcan.socketcan.SocketcanBus>

(3770737, 6221)

In [ ]:
# shm = shared_memory.SharedMemory(create=True, size=1024)	

In [ ]:
# def send_message_wrapper(message: Message, bus: Bus)->None:
timeout = None
done = Event()
# message_to_send
print("before")
process_send = Process(target=send_message, name="Sending CAN messasge", args=(message_to_send, done, bus))
# process_send.start()
# process_send.pid, process_send
# message_to_send

before


In [ ]:
# receive_message(message_proxy, bus)
# process_receive = Process(target=receive_message, name="Receiving CAN message", args=(message_proxy,bus))
# process_receive.start()
# process_receive

In [ ]:
done.set()

In [ ]:
# process_send

In [ ]:
# datetime.fromtimestamp(message_proxy['timestamp']),db.decode_message(message_proxy['arbitration_id'],message_proxy['data'])

In [ ]:
# Process(target=receive_message, args=(message_proxy, bus, done, timeout)).start()

In [ ]:
# try:
# 	recv_message = Message(arbitration_id=recv_message_proxy['arbitration_id'], data=recv_message_proxy['data'], is_extended_id=False)
# except KeyError:
# 	print('KeyError')

In [ ]:
# message = bus.recv(timeout=2)
# if message is not None:
# 	message = can.Message(arbitration_id=example_message.frame_id, data=data, is_extended_id=False) 	
# 	db.decode_message(message.arbitration_id, message.data)
# else:
# 	print('no message received')

In [ ]:
# done.set()

In [ ]:
# import time
# import can
# from can.interfaces.udp_multicast import UdpMulticastBus

# # The bus can be created using the can.Bus wrapper class or using UdpMulticastBus directly
# with can.Bus(channel=UdpMulticastBus.DEFAULT_GROUP_IPv4, interface='udp_multicast') as bus_1:
                
# 	# bus_2 = can.Bus(channel=UdpMulticastBus.DEFAULT_GROUP_IPv6, interface='udp_multicast')
# 	bus_2 = UdpMulticastBus(channel=UdpMulticastBus.DEFAULT_GROUP_IPv4)

#     # register a callback on the second bus that prints messages to the standard out
# 	notifier = can.Notifier(bus_2, [can.Printer()])
	
# 	message = can.Message(arbitration_id=0x123, data=[1, 2, 3])
# 	bus_1.send(message)
# 	# time.sleep(2.0)
# 	# notifier
# 	# msg = bus_2.recv(timeout=1)
# 	# print(msg)


#     # create and send a message with the first bus, which should arrive at the second one
#     # give the notifier enough time to get triggered by the second bus
    
# 	time.sleep(2.0)

#     # msg = bus_2.recv(timeout=2)
#     # give the notifier enough time to get triggered by the second bus
#     # print(msg)
# 	bus_2.shutdown()

In [ ]:
# close and remove vcan0
# !sshpass -v -p  asdf sudo ip link delete vcan0 

!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set down vcan0

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



In [ ]:

!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link delete vcan0 

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 

